### The first of the three gcode .ipynb associated notebooks:  

    gcode_DRO ...
    gcode_Part2.ipynb
    gcode_Part3.ipynb

####  In these gcode notebooks, the cut size (Cut) is reducing the diameter by the Cut size with each roughing pass.

    Ex.  Start diameter = .500
         Cut = .015
         Changing diameter = .500 - .015 = .485



In [ ]:

Turn_Data <- function() {
    StartDia <<- readline("What is Stock Diameter?")
    EndDia <<- readline("What is the Finish Diameter?")
    Cut <<- readline("What is the depth of the cut? Aluminum .015-.020 ")
    Leng <<- readline("What is the length of the travel along the Z axis? s/b from 0 to 2.0 inch range.")
    # Spin <<- readline("What is the Spindle Speed ... max 2800?")       # not used
    # Feed <<- readline("What is the Feedrate ... 4 is slow, 10 fast?")   # not used 
    
    # Convert input strings to numeric values
    StartDia <<- as.numeric(unlist(strsplit(StartDia, ", ")))
    EndDia <<- as.numeric(unlist(strsplit(EndDia, ", ")))
    EndDia <- format(EndDia, nsmall = 3)
    Cut <<- as.numeric(unlist(strsplit(Cut, ", ")))
    Leng <<- as.numeric(unlist(strsplit(Leng, ", ")))
    # Spin <<- as.numeric(unlist(strsplit(Spin, ", ")))
    # Feed <<- as.numeric(unlist(strsplit(Feed, ", ")))
}


# Input:

In [ ]:
Turn_Data()

In [ ]:
X <- "X "
Z <- "Z "

TurnOff <- StartDia - EndDia     # the TOTAL amount of matterial to be removed 
Radius <- StartDia / 2        
Diameter <- StartDia

Reps <- TurnOff / Cut  # To determine the number of cuts to reduce the stock toward finished diameter

# TurnOff    # review values
# Radius
# Diameter
# CutDia
# CutRad
# Reps

####  Fist for-loop.  Used to initialize a df with values for X Z moves during roughing cycle

In [ ]:
# Establish a dataframe w/ X and Z values

if (Leng >= 0)         # to change sign in case neg value entered for Z during prompt
    Leng <- Leng *-1;

ChangingDiameter <- StartDia  # to track the reductions in the diameter 

#Initialize an empty dataframe to store the values
result_df <- data.frame(X = numeric(0), Z = numeric(0))

# Reps is used to 
for (i in 1:Reps) {   
   
    # Create a new row for each iteration with X and Z values and add it to the dataframe
      new_row <- data.frame(X = ChangingDiameter, Z = Leng)
      result_df <- rbind(result_df, new_row)
   
      paste("X", format(ChangingDiameter, nsmall = 3), "Z", format(Leng, nsmall = 2))
    
      ChangingDiameter <- ChangingDiameter - Cut   #  Update the var that tracks the changing X values

    }


# Input:

In [ ]:
# The df created from the for loop
n <- nrow(result_df)
n
result_df$X[n]
# str(result_df)
head(result_df, 3)
tail(result_df, 3)

In [ ]:
# Review the last X value in df - s/b same as (Changing_Diameter), from the X-Z data fram
ChangingDiameter
Reduced_Diameter <- result_df$X[n]
Reduced_Diameter
str(Reduced_Diameter)

### Part 1 - Final roughing cut options. 


In [ ]:
# Add necessary imports and variable definitions here

valid_input <- FALSE    # Boolean var used to control continued entry w/in loop

while (!valid_input) {   
 
    
    Reduced_Diameter <- as.numeric(format(Reduced_Diameter, nsmall = 3))

    LastRoughDia <<- Reduced_Diameter - Cut                # s/be okay to track w/in loop
    Remaining_Uncut_Diameter <<- LastRoughDia - EndDia

    # Prompt the user for input    
    user_input <<- readline(paste("The stock diameter is currently : " , Reduced_Diameter, ". The roughing cut size is:", Cut, ". The final diameter is to be:", format(EndDia, nsmall = 3), ". \n", "One more rough cut will reduce the stock diameter to:", LastRoughDia, " - leaving a diameter to be removed in the final cut decision tree : ", Remaining_Uncut_Diameter, ". \n", "Enter Y to make one last rough cut. Or enter N and go directly to the final cut decision tree."))

    # Convert input to uppercase for case-insensitive comparison
    user_input <- toupper(user_input)

    # Check if the input is 'Y' or 'N'
    if (user_input == 'Y' && Reduced_Diameter >= LastRoughDia) {
        

        Cut <- as.numeric(Cut)
        Reduced_Diameter <- as.numeric(Reduced_Diameter)
        Reduced_Diameter <- Reduced_Diameter - Cut        # Update diameter after the last rough cut  

        # Create a new row for each iteration with X and Z values and add it to the dataframe
        new_row <- data.frame(X = Reduced_Diameter, Z = Leng)            
        # paste("The new diameter is ", new_row)
        Output_gcode_df <<- rbind(result_df, new_row)
              
        print("You entered Yes (Y). Last rough cut was taken and now going to the final cut decision tree")

        valid_input <- TRUE  # Set valid_input to TRUE to exit the loop

      } else if (user_input == 'N') {
        print("You entered No (N).  Moving on to the Finish Cut decision tree.")
        Output_gcode_df <<- result_df
        valid_input <- TRUE  # Set valid_input to TRUE to exit the loop
      } else {
        print("Invalid input. Please enter Y or N.")
        # The loop will continue prompting the user for input
      }
}

# FILEPATH: /c:/Gcode/gcode_DRO-WorkingOriginal.ipynb
# Reduced_Diameter            # var the maintains the changing X values from each roughing turn
# tail(Output_gcode_df, 3)    # this new df replaces result_df. This df will include (rbind) the last rough cut (if performed) 

# nrow(Output_gcode_df)     # test whether the same of not
# nrow(result_df)
    
    

# Input:

In [ ]:
head(Output_gcode_df, 3)

In [ ]:
stopper


## Part - The finish cut(s) options


In [ ]:
valid_input <- FALSE

while (!valid_input) {
 
    
    Reduced_Diameter <- as.numeric(format(Reduced_Diameter, nsmall = 3))
    Remaining_Uncut_Diameter <- Reduced_Diameter - EndDia
    
    
    user_input <<- readline(paste("Welcome to finish cuts.  The current diameter is now:" , Reduced_Diameter, ". The final diameter is to be:",format(EndDia, nsmall = 3), ":  Enter Y or N to enter finish cut dimensions:" ))
    
      # Convert input to uppercase for case-insensitive comparison
      user_input <- toupper(user_input)
    
    print(paste("user_input is : ", user_input, "\n"))
    
    
    if (user_input == 'Y' )   {    

       # condiion: if stock remains to be removed above the EndDia 
       if(!is.na(Remaining_Uncut_Diameter >= 0 )) {
           
            # Prompt the user to input a cut size
            user_input2 <<- readline(paste("The diameter of stock remaining to be removed is:", format(Remaining_Uncut_Diameter, nsmall = 3), "Enter the size of this finish cut (.0xx) :" ))
            FinisCut <- as.numeric(user_input2)
           
           
           if(Reduced_Diameter - FinisCut <= 0) {
               print(paste("Error.  Input makes negative part diameter: ", Reduced_Diameter - FinisCut, " Exiting with error"))
                valid_input <- TRUE
               break
               }

            # Reduced_Diameter <- as.numeric(Reduced_Diameter)
            Reduced_Diameter <- Reduced_Diameter - FinisCut     # Taking finish cut  
           
           format(Reduced_Diameter, nsmall = 3) 
           
           print(paste("Reduced Diameter is now ... ", Reduced_Diameter, ", and FinishCut was .... ", FinisCut, "\n"))            

            # Create a new row for each iteration with X and Z values and add it to the dataframe
             new_row <- data.frame(X = Reduced_Diameter, Z = Leng)           
             paste("The finished stock diameter is to be: ", Reduced_Diameter)
             Output_gcode_df <- rbind(result_df, new_row)
                                      
             Remaining_Uncut_Diameter <- Reduced_Diameter - EndDia
           
           }
           
               if(!is.na(Remaining_Uncut_Diameter >= 0 )) {
           
                    # Prompt the user to input a cut size
                    user_input2 <<- readline(paste("The diameter of stock that remains to be removed is:", format(Remaining_Uncut_Diameter, nsmall = 3), "Enter the size of the first finish cut (.0xx) Do not exceed diameter of stock remaining:" ))
                    FinisCut <- as.numeric(user_input2)

                    # Reduced_Diameter <- as.numeric(Reduced_Diameter)
                    Reduced_Diameter <- Reduced_Diameter - FinisCut     # Taking finish cut  

                   format(Reduced_Diameter, nsmall = 3) 

                   print(paste("Reduced Diameter is now ... ", Reduced_Diameter, ", and FinishCut was .... ", FinisCut, "\n"))


                    # Create a new row for each iteration with X and Z values and add it to the dataframe
                     new_row <- data.frame(X = Reduced_Diameter, Z = Leng)           
                     paste("The finished stock diameter is to be: ", Reduced_Diameter)
                     Output_gcode_df <- rbind(result_df, new_row)

                     Remaining_Uncut_Diameter <- Reduced_Diameter - EndDia
        
        } else if (Remaining_Uncut_Diameter <= 0) {
           print("Another cut would make the final diameter under the desired value")
           valid_input <- TRUE          # Set valid_input to TRUE to exit the loop
           }
        
        
         
       } else if (user_input == 'N') {
        print("You entered No (N)")
        Output_gcode_df <<- result_df
        valid_input <- TRUE                # Set valid_input to TRUE to exit the loop
      } else {
        print("Invalid input. Please enter Y or N.")
        # The loop will continue prompting the user for input
      }
     
      valid_input <- TRUE
   }     
        
print(paste("Exit with gcode file named, gcode_file.csv. Final stock diameter : ", Reduced_Diameter ))
  
#  Restart kernel and run again with any data entry errors  

# Input:

### Restart kernel and run notbooe from the start again with any data entry errors

### Note: the number of repetitions may become important in following notebooks

#### Reps have been saved to a .csv file


In [ ]:
repetitions <- nrow(Output_gcode_df)
repetitions
writeLines(as.character(repetitions), "repetitions.txt")

## Write the df to .csv file

In [ ]:
# Write the updated gcode data to a new .csv file

write.csv(Output_gcode_df, file = "gcode_file.csv", row.names = FALSE)

In [ ]:
# Read the data from the existing .csv file
gcode <- read.csv("gcode_file.csv", header = T)
gcode

## Done !!!

#### Move on to next file: gcode_Part2.ipynb

In [ ]:
stopper  # Used to stop further processing of Notebook cells

#### Code wasteland below


# Example variable
my_variable <- 10

# Display the value of my_variable in the prompt message
user_input <- readline(paste("Enter a value (current value of my_variable is", my_variable, "): "))

# Show the user input
print(user_input)

Reduced_Diameter <- result_df$X[n]
Reduced_Diameter



LastCuts <<- readline(paste("The current diameter is:" , Reduced_Diameter, ": Cut size is:", Cut, "Make one more ", Cut, "sized cut? Enter Y for yes" ))
    





    
    LastCuts <- as.numeric(LastCuts)
    str(LastCuts)
    Reduced_Diameter <- Reduced_Diameter - Cut
    Reduced_Diameter



FinalCut  <- readline(paste("The diameter has been reduced to:", testCutSize, ": Enter value (.0xx) for final cut: "))
     FinalCut <- as.numeric(FinalCut)
FinalCut <- testCutSize - FinalCut
testCutSize




FinalCut 
as.character(FinalCut) 

FinalCut <- as.numeric(unlist(strsplit(FinalCut, ", ")))   # Convert Char to numeric data type
is.numeric(FinalCut)





In [ ]:
testCutSize <- result_df$X[n]


# LastCut <<- readline(paste("The finish cut(s) loop. Diameter is:" , testCutSize, ": Enter next cut size or 0 to quit: "))
    
if (!is.na(FinalCut)) {
    if (FinalCut >= 0 ) {
        print(paste("final cut is: ", FinalCut))
    } else {
        print("FinalCut is something eles")
    }
} else {
    print("wrong input")
    }
   

## Try converting the df to string, 

In [ ]:
#  Adding a column to an existing dataframe

df_X <- cbind(gCode = "G1", Output_gcode_df)
df_X


In [ ]:
vector_X <- df_X$X
vector_Z <- df_X$Z

In [ ]:
vector_X

In [ ]:
# d <- data.frame(x = 1, y = 1:10, char = char))
testDF <- data.frame(gcode = "G01", X = "X", Xg = vector_X, Z = "Z", Zg = vector_Z, F = "F", Speed = Feed)

In [ ]:
string_list <- list()

for(i in 1:nrow(testDF)) {
    row_string <- paste(testDF[i, ], collapse = " ")  # Concatenate row elements into a string
    string_list <- c(string_list, list(row_string))   # Add the row string to the list
    }


In [ ]:
# Need to seperate the list by rows
# Combine the single string with newline characters


combined_string <- paste(string_list, collapse = "\n")
combined_string
is.list(combined_string)

str(combined_string)

In [ ]:
# updated_list <- c(string_list, new_string)

full_string <- c(combined_string, G0_gcode1)

In [ ]:
# Define the file path and name for the output .csv file
file_path <- "gcode_file.csv"

# Write the dataframe to a .csv file
# write.csv(text = combined_string, file = file_path, row.names = FALSE)    # this worked, then it didn't

# Write the string to a .csv file using writeLines()
writeLines(text = full_string, con = file(file_path))

In [ ]:
file_path <- "gcode_file.csv"

# Open the file in append mode using a file connection
file_conn <- file(file_path, open = "a")

# Append the text to the file using writeLines()
writeLines(text = full_string, file_conn)

# Close the file connection
close(file_conn)

In [ ]:
print(paste("X", format(RadDROzero, nsmall = 3), "Z", format(Leng, nsmall = 2)))
    

###  What do I want?  Add G01 to each line.  Add X and Z.  

### This will involve merging or binding columns

In [ ]:
df_X

### Extract the columns

    It's far easier to create a new df from scratch than to modify an existing df

## Build a new G01 data.frame using the X Z vectors

    It far easier to build a df from scratch than to modify an existing df

In [ ]:
testDF    # the new df with all the components for the gcode G01 
nrow(testDF)

##  Unpack the df again

## Looks Right - note this is a list

In [ ]:
string_list
length(string_list)
is.list(string_list)


### This is NOT a list.  Is just a character vector, aka string

## Append the G00 character vector to the G01 character vector

In [ ]:
# Create a new row - the G00 line, to be inserted into the existing string

new_row <- c("G00", "X", ".250", "Z",".75", "F", Feed*2)  # Replace "Data" with your content
new_row
G0_gcode <- paste(new_row, collapse = " ")
G0_gcode
G0_gcode1 <- paste(G0_gcode, collapse = "\n")     
G0_gcode1
str(G0_gcode1)

In [ ]:
full_string
is.list(full_string)
str(full_string)

### The addtion of G00 was easy when using vectors, aka strings, rather than trying to mod a df

### Other code that didn't work out

In [ ]:

# New row of text to insert
new_row <- c("G00", "X", ".250", "Z",".75", "F", Feed*2)  # Replace "Data" with your content
new_row
G0_gcode <- paste(new_row, collapse = " ")
G0_gcode
G0_gcode1 <- paste(G0_gcode, collapse = "\n")
G0_gcode1

In [ ]:
# Write the updated data to a new .csv file
write.csv(G0_gcode1, file = "gcode_file.csv", append = TRUE, row.names = FALSE)  # Replace "updated_data.csv" with your desired file name

In [ ]:
# Determine the position to insert the new row (for example, after the 3rd row)
insert_index <- 2

# Insert the new row into the existing data
updated_data <- rbind(existing_data[1:insert_index, ], 
                new_row, existing_data[(insert_index + 1):nrow(existing_data), ])

In [ ]:
# Your character string
my_string <- "'G01'' ''X''0.235''Z''-0.75'"

# Remove ' character using gsub
cleaned_string <- gsub("'", "", my_string)

# Print the cleaned string
print(cleaned_string)

In [ ]:
testDF[1,]
gcode <- as.character(testDF[1,])
gcode
is.character(gcode)
is.vector(gcode)


In [ ]:
str(cleaned_string)